In [22]:
import numpy as np
from scipy.spatial.distance import cdist

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from numpy import linalg as LA
from scipy.linalg import eigh
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# hyperparameters: Kernel, Regularization, Gamma

In [125]:
# Kernel function

def rbf_kernel(X, sigma, diag=1):
    # Basic SVM with predefined kernel matrix
    N = X.shape[0]
    K = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            if i == j:
                K[i,j] = diag
            else:
                x_i = X[i, :].reshape(1,-1)
                x_j = X[j, :].reshape(1,-1)

                K[i, j] = np.exp(-cdist(x_i, x_j, 'sqeuclidean') / (sigma ** 2)) # euclidean?
    return K


def make_D_matrix(K):
    K_sum = np.sum(K, axis=1)
    D = np.diag(K_sum)

    return D


def make_L_matrix(K, D):
    D_temp = np.diag( np.diag(D) ** -0.5 )
    L = D_temp @ K @ D_temp
    
    w, v = LA.eig(L) # w = eigenvalues, v = normalized (unit “length”) eigenvectors
    
    return L

def step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    w = np.where(w > lambda_cut, 1, 0)
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = 1/L_hat
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)
    
    return L_hat, D_hat, K_hat

def linear_step_transfer(L, k=2):
    w, v = eigh(L)
    lambda_cut = w[-k]
    w = np.where(w >= lambda_cut, w, 0)
    
    L_hat = np.dot(v, np.dot(np.diag(w), v.T))
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ L_hat @ D_hat**(1/2)

    return L_hat, D_hat, K_hat

def polynomial_transfer(L, D, K, t):
    L_hat = L ** t
    D_hat = np.diag(1/np.diag(L_hat))
    K_hat = D_hat**(1/2) @ D**(1/2) @ (LA.inv(D) @ K)**t @ D**(1/2) @ D_hat**(1/2)
    
    print(L_hat)
    print(D_hat)
    print(K_hat)
    return L_hat, D_hat, K_hat
    

def apply_transfer_func(L, D, K, type="linear"):
    if type == "linear":
        return L, D, K
    if type == "step":
        return step_transfer(L)
    if type == "linear_step":
        return linear_step_transfer(L)
    if type == "polynomial":
        return polynomial_transfer(L, D, K, 10)
        
    L_hat = 0
    D_hat = 0
    K_hat = 0
    
    return L_hat, D_hat, K_hat


In [126]:

def accuracy(t, y):
    val = 0.0
    N = len(t)
    for i in range(N):
        if t[i] == y[i]:
            val += 1
    
    return val / N


def test_svm(X, Y):
    # Shuffle data
    np.random.seed(40)
    n_sample = len(X)
    order = np.random.permutation(n_sample)
    X = X[order]
    Y = Y[order].astype(np.float)
    
    # Make Kernel
    K = rbf_kernel(X, 1)
    D = make_D_matrix(K)
    L = make_L_matrix(K, D)
    
    L, D, K = apply_transfer_func(L, D, K, "polynomial")
    
    # Remove data without labels
    K_red = K[:70,:70]
    Y_red = Y[:70]
    
    K_test = K[70:100,:70]
    
    print(K_red.shape)
    
    # Apply to SVM
    clf = SVC(kernel="precomputed", C=1)
    clf.fit(K_red, Y_red)
    
    # Predict and accuracy
    y_pred = clf.predict(K_test)
    
    #print('t', Y[70:100])
    #print('y', y_pred)
    print("accuracy:", accuracy(y_pred, Y[70:100]))
    
    
    # Basic SVM
    clf2 = SVC(kernel="linear", C=1)
    clf2.fit(X[:70], Y_red)
    
    y_pred2 = clf2.predict(X[70:100])
    print("accuracy:", accuracy(y_pred2, Y[70:100]))

In [127]:
# load iris and split data
iris = load_iris()
X = iris.data[:,:2]
y = iris.target

X = X[y != 0]
y = y[y != 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)

test_svm(X,y)

[[  8.79290740e-17   7.08484242e-17   2.83477433e-28 ...,   3.63011210e-40
    9.50712597e-18   8.27494353e-18]
 [  7.08484242e-17   1.27046726e-16   3.09120085e-29 ...,   2.65345041e-41
    1.14278554e-17   1.48387842e-17]
 [  2.83477433e-28   3.09120085e-29   4.73836573e-18 ...,   4.36910187e-19
    9.13673640e-26   2.92558287e-26]
 ..., 
 [  3.63011210e-40   2.65345041e-41   4.36910187e-19 ...,   1.78231322e-14
    1.16398466e-35   3.05147397e-36]
 [  9.50712597e-18   1.14278554e-17   9.13673640e-26 ...,   1.16398466e-35
    7.59547539e-18   8.07476046e-18]
 [  8.27494353e-18   1.48387842e-17   2.92558287e-26 ...,   3.05147397e-36
    8.07476046e-18   1.04848744e-17]]
[[  1.13728026e+16   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   7.87111981e+15   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   2.11043228e+17 ...,   0.00000000e+00
    0.00000000e+00   